In [4]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, Input, MaxPooling2D, Add, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model


In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [6]:
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

#### Create Identity block

In [7]:
def identity_block(x, filters):
    f1, f2 = filters
    X_shortcut = x

    # First component of the main path
    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)

    # Second component of the main path
    x = Conv2D(f2, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)

    # Add the shortcut connection to the main path
    x = Add()([x, X_shortcut])
    x = Activation('relu')(x)
    return x

    

#### Create Convolutional block

In [8]:
def convolution_block(x, filters, s=2):
    f1, f2 = filters
    X_shortcut = x

    # First component of the main path
    x = Conv2D(f1, kernel_size=(3, 3), strides=(s, s), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    
    # Second component of the main path
    x = Conv2D(f2, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    
    # Shortcut path (match dimensions)
    X_shortcut = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='same', activation='relu')(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)
    
    # Merge the paths
    x = Add()([x, X_shortcut])    
    x = Activation('relu')(x)
    return x

#### Design ResNet Architecture

In [9]:
def ResNet(input_shape, num_classes):
    inputs = Input(input_shape) 
    
    # Stage-1 
    x = Conv2D(64, (5, 5), strides=(2, 2), name='conv1')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    # Stage-2
    x = convolution_block(x, filters=[64, 64], s=1)
    x = identity_block(x, filters=[64, 64])

    # Stage-3
    x = convolution_block(x, filters=[128, 128], s=2)
    x = identity_block(x, filters=[128, 128])
    
    # Stage-4
    x = convolution_block(x, filters=[256, 256], s=2)
    x = identity_block(x, filters=[256, 256]) 

    # Stage-5
    x = convolution_block(x, filters=[512, 512], s=2)
    x = identity_block(x, filters=[512, 512])
     
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model
     

In [10]:
model = ResNet((32, 32, 3), num_classes=10)

In [11]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1 (Conv2D)      │ (None, 14, 14,    │      4,864 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 14, 14,    │        256 │ conv1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 14, 14,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 6, 6, 64)  │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 6, 6, 64)  │     36,928 │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 6, 64)  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 6, 6, 64)  │     36,928 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 6, 6, 64)  │      4,160 │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 6, 64)  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 6, 64)  │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 6, 6, 64)  │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 6, 6, 64)  │          0 │ add[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 6, 6, 64)  │     36,928 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 6, 64)  │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 6, 6, 64)  │     36,928 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 6, 64)  │        256 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 6, 6, 64)  │          0 │ batch_normalizat… │
│                     │                   │            │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 11,195,850 (42.71 MB)

 Trainable params: 11,186,122 (42.67 MB)

 Non-trainable params: 9,728 (38.00 KB)

In [12]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train, 
          epochs=10, 
          batch_size=32)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 212s 126ms/step - accuracy: 0.3166 - loss: 2.0196
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 200s 128ms/step - accuracy: 0.5210 - loss: 1.3649
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 203s 130ms/step - accuracy: 0.5967 - loss: 1.1741
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 208s 133ms/step - accuracy: 0.6443 - loss: 1.0502
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 206s 132ms/step - accuracy: 0.6844 - loss: 0.9288
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 205s 131ms/step - accuracy: 0.7047 - loss: 0.8634
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 206s 132ms/step - accuracy: 0.7508 - loss: 0.7309
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 206s 132ms/step - accuracy: 0.7636 - loss: 0.6835
Epoch 9/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 208s 133ms/step - accuracy: 0.8052 - loss: 0.5774
Epoch 10/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 207s 132ms/step - accuracy: 0.8288 - loss: 0.5039


In [14]:
model.evaluate(x_test, y_test, verbose=0)

[35.517215728759766, 0.41040000319480896]